In [66]:
import os
import logging

import pandas as pd
import tensorflow.keras as keras

from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import plot_model

# Log setting
logging.basicConfig(format="%(asctime)s %(levelname)s %(message)s", datefmt="%H:%M:%S", level=logging.INFO)

In [67]:
PROCESSED_DIR_PATH = "datasets"
MODEL_DIR_PATH = "model"

In [68]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from sklearn.metrics import classification_report
from sklearn.preprocessing import MinMaxScaler


def preprocessing(df: pd.DataFrame):
    # Shuffle the dataset
    df = df.sample(frac=1)

    # Split features and labels
    x = df.iloc[:, df.columns != 'Label']
    y = df[['Label']].to_numpy()

    # Scale the features between 0 ~ 1
    scaler = MinMaxScaler()
    x = scaler.fit_transform(x)

    return x, y


def reshape_dataset_cnn(x: np.ndarray) -> np.ndarray:
    # Add padding columns
    result = np.zeros((x.shape[0], 81))
    result[:, :-3] = x

    # Reshaping dataset
    result = np.reshape(result, (result.shape[0], 9, 9))
    result = result[..., tf.newaxis]
    return result


def plot_history(history: tf.keras.callbacks.History):
    # summarize history for accuracy
    plt.plot(history.history['sparse_categorical_accuracy'])
    plt.plot(history.history['val_sparse_categorical_accuracy'])
    plt.title('model2 accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model2 loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()


def evaluation(model: keras.Model, x_test: np.ndarray, y_test: np.ndarray):
    score = model.evaluate(x_test, y_test, verbose=False)
    logging.info('Evaluation:\nLoss: {}\nAccuracy : {}\n'.format(score[0], score[1]))

    # F1 score
    y_pred = model.predict(x_test, batch_size=1024, verbose=False)
    y_pred = np.argmax(y_pred, axis=1)

    logging.info("\n{}".format(classification_report(y_test, y_pred)))

In [69]:
def create_cnn_model() -> keras.Model:
    # Creating layers
    inputs = keras.layers.Input(shape=(9, 9, 1))
    x = keras.layers.Conv2D(120, 2, activation='relu', padding="same")(inputs)
    x = keras.layers.MaxPooling2D(2)(x)
    x = keras.layers.Conv2D(60, 3, activation='relu', padding="same")(x)
    x = keras.layers.MaxPooling2D(2)(x)
    x = keras.layers.Conv2D(30, 4, activation='relu', padding="same")(x)
    x = keras.layers.Reshape((2, 2*30))(x)
    x = keras.layers.GRU(64, return_sequences=True)(x)
    x = keras.layers.GRU(32)(x)
    x = keras.layers.Flatten()(x)
    outputs = keras.layers.Dense(15, activation='softmax')(x)
    cnn_model = keras.Model(inputs=inputs, outputs=outputs, name='cnn')

    # Compile layers
    cnn_model.compile(loss='sparse_categorical_crossentropy',
                      metrics=['sparse_categorical_accuracy'],
                      optimizer='adam')

    return cnn_model

In [70]:
# Create model
model = create_cnn_model()
logging.info(model.summary())

Model: "cnn"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_15 (InputLayer)       [(None, 9, 9, 1)]         0         
                                                                 
 conv2d_42 (Conv2D)          (None, 9, 9, 120)         600       
                                                                 
 max_pooling2d_24 (MaxPoolin  (None, 4, 4, 120)        0         
 g2D)                                                            
                                                                 
 conv2d_43 (Conv2D)          (None, 4, 4, 60)          64860     
                                                                 
 max_pooling2d_25 (MaxPoolin  (None, 2, 2, 60)         0         
 g2D)                                                            
                                                                 
 conv2d_44 (Conv2D)          (None, 2, 2, 30)          28830   

15:35:46 INFO None


In [74]:
# Checkpoint
cp_path = os.path.join(MODEL_DIR_PATH, "5_2_cnn_weights-improvement-{epoch:02d}-{sparse_categorical_accuracy:.2f}.hdf5")
checkpoint = ModelCheckpoint(cp_path, monitor='sparse_categorical_accuracy', verbose=1,
                             save_best_only=True, mode='max')
callbacks_list = [checkpoint]

# Training
df = pd.read_csv(os.path.join(PROCESSED_DIR_PATH, 'train_MachineLearningCVE.csv'), skipinitialspace=True)
logging.info("Class distribution\n{}".format(df.Label.value_counts()))

16:10:23 INFO Class distribution
0     1771379
4      184858
10     108072
2      102422
3        8234
7        6350
11       4717
6        4637
5        4399
1        1573
12       1206
14        522
9          29
13         17
8           9
Name: Label, dtype: int64


In [75]:
X, Y = preprocessing(df)
del df
X = reshape_dataset_cnn(X)

In [76]:
# Training
logging.info("*** TRAINING START ***")
history = model.fit(X, Y, validation_split=0.1, epochs=125, batch_size=1024, verbose=True)

16:10:41 INFO *** TRAINING START ***


Epoch 1/125
1933/1933 [==============================] - 216s 112ms/step - loss: 0.0552 - sparse_categorical_accuracy: 0.9762 - val_loss: 0.0527 - val_sparse_categorical_accuracy: 0.9795
Epoch 2/125
1933/1933 [==============================] - 215s 111ms/step - loss: 0.0449 - sparse_categorical_accuracy: 0.9820 - val_loss: 0.0430 - val_sparse_categorical_accuracy: 0.9827
Epoch 3/125
1933/1933 [==============================] - 206s 107ms/step - loss: 0.0418 - sparse_categorical_accuracy: 0.9833 - val_loss: 0.0472 - val_sparse_categorical_accuracy: 0.9818
Epoch 4/125
1933/1933 [==============================] - 201s 104ms/step - loss: 0.0386 - sparse_categorical_accuracy: 0.9847 - val_loss: 0.0398 - val_sparse_categorical_accuracy: 0.9862
Epoch 5/125
1933/1933 [==============================] - 200s 103ms/step - loss: 0.0367 - sparse_categorical_accuracy: 0.9856 - val_loss: 0.0353 - val_sparse_categorical_accuracy: 0.9868
Epoch 6/125
1933/1933 [==============================] - 200s 103

KeyboardInterrupt: 

In [ ]:
logging.info("*** TRAINING FINISH ***")
del X, Y

In [ ]:
# Evaluation
df = pd.read_csv(os.path.join(PROCESSED_DIR_PATH, 'test_MachineLearningCVE.csv'), skipinitialspace=True)
logging.info("Class distribution\n{}".format(df.Label.value_counts()))

In [ ]:
X, Y = preprocessing(df)
del df
X = reshape_dataset_cnn(X)

In [ ]:
evaluation(model, X, Y)
del X, Y

In [ ]:
logging.info("*** END ***")